In [ ]:
'''

  train_state_v1
    
'''

from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np
import os, sys
import copy
import imageio
import scipy.io as io
import random
import skimage
import pandas as pd
import shutil
import pickle

from glob import glob
from random import shuffle
from datetime import datetime

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Conv3D, MaxPooling3D, Activation, BatchNormalization, Flatten, Dense, Dropout
from keras.preprocessing.image import ImageDataGenerator

from keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from sklearn.utils import class_weight
from skimage.transform import rescale, resize, downscale_local_mean
from tqdm import tqdm
from tlearn import tlearn

import keras.backend as K


with open('train_data_classwise.pickle', 'rb') as f:
    dat = pickle.load(f)
    
print(dat.keys())

print(len(dat['Xc_img']))
print(len(dat['yc_class']))
print(len(dat['yc_state']))

Xc_img = dat['Xc_img']
yc_class = dat['yc_class']
yc_state = dat['yc_state']

class_set = yc_class.keys()
state_set = yc_state.keys()

print(class_set)

In [ ]:
history = {}
state_name_by_class = {}



for key in tqdm(yc_class):
    
    #print(key)
    if history.get(key) is None:
        history[key] = list()
        state_name_by_class[key] = list()
    X1_list = Xc_img[key]; y1_list = yc_state[key]
    print(len(X1_list), len(y1_list))
    X1 = np.stack(X1_list, axis=0); y1 = np.stack(y1_list, axis=0); state_name = np.unique(y1)
    print(f'key={key}\nstate_name={state_name}')
    y_num = 200 * np.ones((y1.shape[0]))
    nstate = state_name.shape[0]
    for k in range(nstate):
        index = np.where( y1 == state_name[k] )
        y_num[index] = k
    y1_1hot = to_categorical(y_num)

    # For oversampling
    x_aug_arr = np.array([], dtype=np.uint8).reshape(0,256,256,3)
    y_aug_arr = np.array([], dtype=np.uint8).reshape(0,nstate) # one-hot
    
    if False:
        count = 0
        plt.figure(figsize=(15, 15))
        for jj in range(0, 80, 5):
            plt.subplot(4, 4,count+1)
            plt.imshow(X1[jj, :, :, :])
            # plt.title(f'{y1_1hot[jj, :]}')
            plt.title(f'key={key}\nstate={y1[jj]}')
            count += 1
        plt.show()

    if True:
        # Class imbalance 문제 해결. Image oversampling by augmentation
        nimg_state_list = []
        for ss in range(nstate):
            a = np.where(y_num == ss)[0]
            nimg_state_list.append(a.shape[0])
        print(f'state-by-state : num of images = {nimg_state_list}')
        oversampling_factors = np.max(nimg_state_list)/ np.array(nimg_state_list)
        print(f'oversampling factors = {oversampling_factors}')
        ind_good = np.argmax(nimg_state_list)
        
        for jj in range(len(X1)):
            
            if y_num[jj]==ind_good:
                continue
                
            else:
                x1 = X1[jj,:,:,:]
                x1 = np.expand_dims(x1, 0)
                datagen = ImageDataGenerator(rotation_range=90, width_shift_range=0.2, height_shift_range=0.2,horizontal_flip=False)
                osf = int(oversampling_factors[ int(y_num[jj])])
                it = datagen.flow(x1, batch_size= 1)
                # print(jj, int(y_num[jj]))
                # print(oversampling_factors[ int(y_num[jj]) ])
                img_os_list = []
                y_num_os_list = []
                y_num_1hot = to_categorical(y_num[jj], num_classes=nstate)
                #print(y_num_1hot)
                for kk in range(osf):
                    batch = it.next()
                    img = batch[0].astype('uint8') # 256 x 256 x 3
                    # print(img.shape) 
                    img_os_list.append(img)
                    y_num_os_list.append(y_num_1hot)
                # print(batch.shape)
                img_os_arr = np.stack(img_os_list, axis=0)
                y_num_os_arr = np.stack(y_num_os_list, axis=0)
                # print(nstate, y_num_os_arr)
                x_aug_arr = np.concatenate((x_aug_arr, img_os_arr), axis=0)
                # print(y_1hot_os_arr.shape, type(y_1hot_os_arr))
                y_aug_arr = np.concatenate((y_aug_arr, y_num_os_arr), axis=0)
                # print(img_os_arr.shape, x_aug_arr.shape, y_aug_arr.shape)
        
        X1 = np.concatenate( (X1, x_aug_arr), axis=0 )
        y1_1hot = np.concatenate( (y1_1hot, y_aug_arr), axis=0 )
        print(f'X1: {X1.shape}, y1_1hot: {y1_1hot.shape}')
    
    if True:
        # 각각의 클래스에 대해 학습을 진행. 모델 저장.
        history[key].append( tlearn(X1, y1_1hot, y_num, key, nstate) )

    # 각각의 클래스에 대해 state 이름 순서 저장. 나중에 test 데이터 예측시 이용할 계획.
    state_name_by_class[key].append(state_name)
    
if True:
    with open('state_name_by_class.pickle', 'wb') as f:
        pickle.dump(state_name_by_class, f, pickle.HIGHEST_PROTOCOL)  
    
    